#### Forecast Crimes in Chicago
+ load 'chicagoCrimesByYear.csv' from the course git repository (this data has been extraction and aggregated from Chicago Open Data Portal

+ Dataset properties
    + Every Row is a year-month
    + every rolumn is a count of unique crime ids
    
    
##### Deliverables

+ Get print the maximum and minimun date in the data set
+ plot the count of homicides by year/month
+ Create use a row shift by one month to predict homicides in one month, with all crime data from the prior month
 (notes: use iloc to select)
+ Split the data by leaving the last year out as a test set ( use the pandas data frame index to select)
+ train a linear model that predicts the count of homicides in the next month using sklearn linear regression
+ calcuate r2_score, mean_absolute_error and explained varience using sklearn metrics
+ create a bar plot, sorted by coefficient of the linear models coefficients
+ create a scatter plot of predictions vs 


+ Answer these questions:
  * If 10 robberies happen this month, how maney homicides you expect in the following month ?

  * If 10 stalkings happen this month, how maney homicides you expect in the following month?
 

+ write a paragraph summary of findings

##### More Information 

http://gis.chicagopolice.org/CLEARMap_crime_sums/crime_types.html





In [64]:
import pandas as pd
from matplotlib import pyplot as plt
path = 'chicagoCrimesByYear.csv'
df = pd.read_csv(path)
# create a column withs days (always the first day of the month)

df['day'] = 1

# reindex the data frame so each row is year_month-day
df.index = pd.to_datetime(df[['year', 'day', 'month']])

# drop year month day columns
df.drop(['year', 'day', 'month'], inplace=True, axis=1)

df.head()

,ARSON,ASSAULT,BATTERY,BURGLARY,CONCEALED CARRY LICENSE VIOLATION,CRIM SEXUAL ASSAULT,CRIMINAL DAMAGE,CRIMINAL TRESPASS,DECEPTIVE PRACTICE,DOMESTIC VIOLENCE,...,OTHER OFFENSE,PROSTITUTION,PUBLIC INDECENCY,PUBLIC PEACE VIOLATION,RITUALISM,ROBBERY,SEX OFFENSE,STALKING,THEFT,WEAPONS VIOLATION
2001-01-01,67.0,2123.0,6525.0,1934.0,0.0,163.0,3966.0,1191.0,1393.0,1.0,...,2800.0,563.0,0.0,161.0,2.0,1396.0,218.0,26.0,7866.0,337.0
2001-02-01,57.0,2029.0,6040.0,1666.0,0.0,135.0,3664.0,1063.0,1159.0,0.0,...,2487.0,426.0,1.0,173.0,0.0,1159.0,149.0,13.0,6669.0,301.0
2001-03-01,93.0,2824.0,7658.0,1832.0,0.0,153.0,4615.0,1141.0,1308.0,0.0,...,2836.0,550.0,0.0,267.0,2.0,1399.0,184.0,17.0,7765.0,344.0
2001-04-01,89.0,2746.0,8325.0,1931.0,0.0,133.0,4922.0,1133.0,1197.0,0.0,...,2483.0,564.0,1.0,229.0,1.0,1341.0,169.0,29.0,7702.0,321.0
2001-05-01,94.0,2903.0,8886.0,1997.0,0.0,156.0,4756.0,1067.0,1255.0,0.0,...,2529.0,503.0,1.0,239.0,1.0,1491.0,225.0,11.0,8417.0,390.0


In [73]:
label_col = 'HOMICIDE'


# example of data shift by 1
n_ahead = 1
y = df.iloc[0:-n_ahead ][label_col].fillna(0).values
x = df.iloc[n_ahead :, :].fillna(0).values
x.shape, y.shape

feature_names = list(df.columns)

In [79]:

# using stats model API (NOTE Home requires sklearn API)
from statsmodels.api import GLM

#
m = GLM(exog = x, endog=y).fit()
m.summary()

coefs = m._results.params
pvals = m.pvalues
results = pd.DataFrame({'p': pvals, 'coef': coefs}, index = feature_names)


# filter out high p values (greater than .01)
results  = results.loc[results['p'] < .1]

results = results.sort_values(by='coef', ascending = True)
results

,p,coef
OFFENSE INVOLVING CHILDREN,0.005274,-0.063845
DECEPTIVE PRACTICE,0.077475,0.009487
ROBBERY,0.036641,0.018835
GAMBLING,0.064405,0.049706
SEX OFFENSE,0.081122,0.080169
CRIM SEXUAL ASSAULT,0.043319,0.100167
HOMICIDE,0.008434,0.203308
STALKING,0.026554,0.333977
NON-CRIMINAL,0.006027,1.764795
HUMAN TRAFFICKING,0.080106,1.911063
